In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from types import ModuleType
import sys
import io
import traceback

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct")

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
input_text = "#write a quick sort algorithm"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


#write a quick sort algorithm in python


def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less = [i for i in arr[1:] if i <= pivot]
        greater = [i for i in arr[1:] if i > pivot]
        return quick_sort(less) + [pivot] + quick_sort(greater)

#test the function
print(quick_sort([3,6,8,10,


In [ ]:
from datasets import load_dataset
human_eval = load_dataset("openai_humaneval")['test']

In [ ]:
def split_problem_tests(problem):
    pre_base_str, tests = problem["test"].split("def check(candidate):\n")
    base_str = "def check(candidate):\n"
    split_tests = []
    # NOTE: assumes human-eval-specific logic for multiline asserts & for-loops
    # won't work properly if multiline assert nested within for-loop
    multiline_assert, parts = False, []
    for_loop, fl_parts = False, []
    for i in tests.split("\n"):
        if multiline_assert:
            parts.append(i)
            if i.lstrip().startswith("]"):
                test = "\n".join(parts)
                split_tests.append(pre_base_str + base_str + test)
                multiline_assert = False
        elif i.lstrip().startswith("assert") and i.lstrip()[-1] == "[":
            multiline_assert = True
            parts = [i]
        elif for_loop:
            fl_parts.append(i)
            if i.lstrip().startswith("assert"):
                test = "\n".join(fl_parts)
                split_tests.append(pre_base_str + base_str + test)
                for_loop, fl_parts = False, []
        elif (
            (i.lstrip() == "")
            or (i.lstrip().startswith("#"))
            or (i.lstrip().startswith("print"))
        ):
            continue
        elif not (i.lstrip().startswith("assert")):
            fl_parts.append(i)
            if i.lstrip().startswith("for"):
                for_loop = True
        # special logic for HumanEval/151
        elif problem["task_id"] == "HumanEval/151" and (
            i.lstrip().startswith("assert candidate(lst)")
        ):
            fl_parts.append(i)
            test = "\n".join(fl_parts)
            split_tests.append(pre_base_str + base_str + test)
            fl_parts = []
        else:
            split_tests.append(pre_base_str + base_str + i)
    return split_tests

In [ ]:
def evaluate_code(task, my_solution=None):
    """
    Given a task from HumanEval, evaluate the correctness of a code generation.

    Args:
        task (dict): A dictionary containing a HumanEval task with the following keys:
                     - task_id: Unique identifier for the task.
                     - prompt: The Python function prompt (with docstring).
                     - canonical_solution: The reference solution.
                     - test: The unit tests for this function.

    Returns:
        result (dict): A dictionary containing the task_id, prompt, and whether the solution passed the tests.
    """

    task_id = task["task_id"]
    prompt = task["prompt"]
    testcases = split_problem_tests(task)
    generated_func_name = task["entry_point"]

    # Combine the prompt with the canonical solution
    if my_solution is None or not isinstance(my_solution, str):
        raise ValueError(f"solution code provided incorrectly: {my_solution}")
    full_code = prompt + "\n" + my_solution

    # Create a temporary module to execute the code
    module = ModuleType("temp_module")
    sys.modules["temp_module"] = module

    # Redirect stdout to capture test results
    saved_stdout = sys.stdout
    saved_stderr = sys.stderr
    stdout = io.StringIO()
    stderr = io.StringIO()
    sys.stdout = stdout
    sys.stderr = stdout

    num_asserts, num_correct, num_failed = len(testcases), 0, 0
    errors = []
    successes = []
    try:
        # Execute the function and test code
        # print(full_code)
        exec(full_code, module.__dict__)

        # print(f"func {generated_func_name} not defined!" if "has_close_elements" not in module.__dict__ else module.__dict__["has_close_elements"])

        # test_code = test_code + "\n" + f"check({generated_func_name})"
        # print(test_code)
        # exec(test_code, module.__dict__)

        # Evaluate each assert independently
        for testcase in testcases:
            print(testcase)
            try:
                testcase = testcase + "\n" + f"check({generated_func_name})"
                exec(testcase, module.__dict__)

                # If no error occurs
                num_correct += 1
                successes.append("True")
            except AssertionError as e:
                # If assertion error occurs, the testcase fails
                num_failed += 1
                err_str = f"AssertionError: {e}"
                errors.append(err_str)
                successes.append("False")
            except Exception as e:
                num_failed += 1
                err_str = f"{type(e).__name__}: {e}"
                errors.append(err_str)
                successes.append("False")

        assert num_asserts == (num_correct + num_failed), "num_asserts != (num_correct + num_failed)   %s" % ((num_asserts, num_correct, num_failed),)
        is_success = (num_correct == num_asserts)
        success_rate = (num_correct / num_asserts)

        error = "\n".join(errors)

    except AssertionError as e:
        raise
    except Exception as e:
        err_str = f"{type(e).__name__}: {e}"
        print(err_str, file=stderr)

        is_success = False
        success_rate = 0
        error = f"Compilation error:\n {err_str}"
    finally:
        # Reset stdout and cleanup
        sys.stdout = saved_stdout
        sys.stderr = saved_stderr
        del sys.modules["temp_module"]

    result = {"task_id": task_id, "prompt": prompt, "evaluated_code": my_solution,
              "success": is_success, "success_rate": success_rate, "num_correct_testcases": num_correct, "num_total_testcases": num_asserts,
              "success_map": successes, "error": error}

    return result

In [ ]:
problem = human_eval[0]["prompt"]
prompt = '''
Please continue to complete the function. You are not allowed to modify the given code and do the completion only. Please return all completed function in a codeblock. Here is the given code to do completion:
```{}
{}
```
'''.strip().format("python", problem.strip())

In [ ]:
chat = [{'role': 'user', 'content': prompt }]
inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True)

In [ ]:
stop_id = tokenizer.convert_tokens_to_ids("<|EOT|>")

In [ ]:
outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=False,
        # top_p=0.95,
        # temperature=temperature,
        pad_token_id=stop_id,
        eos_token_id=stop_id
    )

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
print(output)

Here is the completed function:

```python
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    numbers.sort()
    for i in range(1, len(numbers)):
        if numbers[i] - numbers[i - 1] < threshold:
            return True
    return False
```

This function works by first sorting the list of numbers. Then it iterates over the sorted list, checking if the difference between each number and the previous one is less than the threshold. If it finds such a pair, it returns True. If it doesn't find any such pair after checking all numbers, it returns False.



In [ ]:
result = output.split("```python")[1].split("```")[0]

In [ ]:
evaluate_code(human_eval[0], result)

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'evaluated_code': '\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if numbers[i] - numbers[i - 1] < threshold:\n            return True\n    return False\n',
 'success': True,
 'success_rate': 1.0,
 'num_correct_testcases

In [ ]:
prompt = '''Please explain this python code\n```{}\n{}\n```'''.strip().format("python", result)
chat = [{'role': 'user', 'content': prompt }]
inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True)
stop_id = tokenizer.convert_tokens_to_ids("<|EOT|>")

In [ ]:
outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=False,
        # top_p=0.95,
        # temperature=temperature,
        pad_token_id=stop_id,
        eos_token_id=stop_id
    )
output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [ ]:
print(output)

This Python function `has_close_elements` checks if there are any two numbers in a given list `numbers` that are closer to each other than a given threshold.

Here's a step-by-step explanation:

1. The function takes two arguments: a list of numbers `numbers` and a threshold `threshold`. The list is expected to contain floats.

2. The function sorts the list of numbers in ascending order.

3. It then iterates over the sorted list, comparing each number with the previous one.

4. If the difference between the current number and the previous one is less than the threshold, the function immediately returns `True`, indicating that there are two numbers in the list that are closer to each other than the threshold.

5. If the function iterates over the entire list without finding any pair of numbers that are closer to each other than the threshold, it returns `False`.

In the examples you provided, the function is used to check if there are any two numbers in the list that are closer to each

In [ ]:
prompt = """Write a solution to the following problem: \n```python\n{}\n```\n\nYour solution is required to use exactly the same function name(s) as described in the problem above.\n\n""".format(output)
chat = [{'role': 'user', 'content': prompt }]
inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True)
stop_id = tokenizer.convert_tokens_to_ids("<|EOT|>")

In [ ]:
outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=False,
        # top_p=0.95,
        # temperature=temperature,
        pad_token_id=stop_id,
        eos_token_id=stop_id
    )
output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [ ]:
result = output.split("```python")[1].split("```")[0]
evaluate_code(human_eval[0], result)

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'evaluated_code': '\ndef has_close_elements(numbers, threshold):\n    numbers.sort()\n    for i in range(1, len(numbers)):\n        if numbers[i] - numbers[i-1] < threshold:\n            return True\n    return False\n',
 'success': True,
 'success_rate': 1.0,
 'num_correct_testcases': 7,
 'num_total_testcases': 7,
 'success_map': ['True', 'True', 'True', 'True', 'True', 'True', 'True'],
 'error': ''}

In [ ]:
explanation_map = {}
code_map = {}
new_code_map = {}
task_success_map = {}
new_task_success_map = {}

In [ ]:
for i in range(164):

  problem = human_eval[i]["prompt"]
  prompt = '''\nPlease continue to complete the function. You are not allowed to modify the given code and do the completion only. Please return all completed function in a codeblock. Here is the given code to do completion:\n```{}\n{}\n```'''.strip().format("python", problem.strip())

  chat = [{'role': 'user', 'content': prompt }]
  inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True)
  stop_id = tokenizer.convert_tokens_to_ids("<|EOT|>")

  outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=False,
        # top_p=0.95,
        # temperature=temperature,
        pad_token_id=stop_id,
        eos_token_id=stop_id
    )
  output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  code = output.split("```python")[1].split("```")[0]
  code_map[i] = code

  results = evaluate_code(human_eval[i], code)
  task_success_map[i] = results["success_map"]

  prompt = '''Please explain this python code\n```{}\n{}\n```'''.strip().format("python", code)
  chat = [{'role': 'user', 'content': prompt }]
  inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True)
  stop_id = tokenizer.convert_tokens_to_ids("<|EOT|>")

  outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=False,
        # top_p=0.95,
        # temperature=temperature,
        pad_token_id=stop_id,
        eos_token_id=stop_id
    )
  explanation = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  explanation_map[i] = explanation

  prompt = """Write a solution to the following problem: \n```python\n{}\n```\n\nYour solution is required to use exactly the same function name(s) as described in the problem above. Please return all completed function in a codeblock. \n\n""".format(explanation)
  chat = [{'role': 'user', 'content': prompt }]
  inputs = tokenizer.apply_chat_template(chat, return_tensors="pt", add_generation_prompt=True)
  stop_id = tokenizer.convert_tokens_to_ids("<|EOT|>")

  outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=False,
        # top_p=0.95,
        # temperature=temperature,
        pad_token_id=stop_id,
        eos_token_id=stop_id
    )
  output = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
  new_code = output.split("```python")[1].split("```")[0]
  new_code_map[i] = new_code

  results = evaluate_code(human_eval[i], new_code)
  new_task_success_map[i] = results["success_map"]
  print(new_task_success_map[i])
  print(f"Finished Evaluating Task {i}")

In [ ]:
num_correct = 0
for i in range(107):
  if task_success_map[i] == new_task_success_map[i]:
    num_correct += 1
print(num_correct)

In [ ]:
print(task_success_map)
print(explanation_map)
print(new_task_success_map)